## First part of this challenge is to make sure our data is clean and able to used for analysis. 

In [1]:
import pandas as pd
import gzip
import json
import shutil
import os
import numpy as np
from datetime import datetime
from ast import literal_eval
from sqlalchemy import create_engine

In [2]:
for i in os.listdir():
    if 'json' in i:
        with gzip.open(i, 'rb') as f1:
              with open(i.replace('.gz',''), 'wb') as f2:
                shutil.copyfileobj(f1, f2)

In [3]:
df_users = pd.read_json("users.json", lines=True)
df_receipts = pd.read_json("receipts.json", lines=True)
df_brands = pd.read_json("brands.json", lines=True)

### Users Dataframe

In [4]:
df_users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [5]:
len(df_users)


495

In [6]:
df_users.signUpSource.value_counts()

Email     443
Google      4
Name: signUpSource, dtype: int64

In [7]:
df_users.state.value_counts()

WI    396
NH     20
AL     12
OH      5
IL      3
CO      1
KY      1
SC      1
Name: state, dtype: int64

In [8]:
df_users.isnull().sum()

_id              0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64

In [9]:
df_users.isnull().sum()

_id              0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64

#### Taking a look at the users file we can see that the columns _id, createdDate, and lastLogin needed to be cleaned up and the createdDate and lastLogin columns need to be converted to datetime.

In [10]:
# Function to get the 2nd element from the dictionary in our columns.

def parse(df, column, key_word_parse):
    df[column] = df[column].apply(lambda x: x[key_word_parse])
    

In [11]:
parse(df_users,'_id','$oid')

In [12]:
df_users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [13]:
#We want to convert UTC time to local time and ignore the missing values in our columns.
#User previous Github Repo for help!

def convert_date(x):
    try:
        return(datetime.utcfromtimestamp(int(x['$date'])/1000).strftime('%Y-%m-%d %H:%M:%S'))
    except TypeError:
        return(None)


In [14]:
df_users['createdDate'] = df_users['createdDate'].apply(lambda x: convert_date(x))


In [15]:
df_users['lastLogin'] = df_users['lastLogin'].apply(lambda x: convert_date(x))

In [16]:
df_users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI


### Brands Dataframe

In [17]:
df_brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [18]:
parse(df_brands, '_id', '$oid')

In [19]:
df_brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [20]:
df_brands.isnull().sum()

_id               0
barcode           0
category        155
categoryCode    650
cpg               0
name              0
topBrand        612
brandCode       234
dtype: int64

In [21]:
df_brands['cpg'].values

array([{'$id': {'$oid': '601ac114be37ce2ead437550'}, '$ref': 'Cogs'},
       {'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, '$ref': 'Cogs'},
       {'$id': {'$oid': '601ac142be37ce2ead437559'}, '$ref': 'Cogs'}, ...,
       {'$ref': 'Cogs', '$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}},
       {'$ref': 'Cogs', '$id': {'$oid': '5332f5f6e4b03c9a25efd0b4'}},
       {'$id': {'$oid': '6026d757be37ce6369301467'}, '$ref': 'Cogs'}],
      dtype=object)

#### Seems like the cpg column has multiple values inside the dictionary. Let's extract them and create new columns for the key and values.

In [22]:
df_brands['cpg_id'] = df_brands['cpg'].apply(lambda x: x['$id']['$oid'])

In [23]:
df_brands['cpg_ref'] = df_brands['cpg'].apply(lambda x: x['$ref'])

In [24]:
df_brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode,cpg_id,cpg_ref
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN,601ac114be37ce2ead437550,Cogs
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,601ac142be37ce2ead437559,Cogs
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,5332fa12e4b03c9a25efd1e7,Cogs


### Receipts Dataframe

In [25]:
df_receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [26]:
df_receipts.isnull().sum()

_id                          0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate               551
modifyDate                   0
pointsAwardedDate          582
pointsEarned               510
purchaseDate               448
purchasedItemCount         484
rewardsReceiptItemList     440
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
dtype: int64

In [27]:
len(df_receipts)

1119

In [28]:
parse(df_receipts, '_id', '$oid')


In [29]:
df_receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [30]:
df_receipts['createDate'] = df_receipts['createDate'].apply(lambda x: convert_date(x))
df_receipts['dateScanned'] = df_receipts['dateScanned'].apply(lambda x: convert_date(x))
df_receipts['finishedDate'] = df_receipts['finishedDate'].apply(lambda x: convert_date(x))
df_receipts['modifyDate'] = df_receipts['modifyDate'].apply(lambda x: convert_date(x))
df_receipts['pointsAwardedDate'] = df_receipts['pointsAwardedDate'].apply(lambda x: convert_date(x))
df_receipts['purchaseDate'] = df_receipts['purchaseDate'].apply(lambda x: convert_date(x))

In [31]:
df_receipts

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,None,2021-01-03 15:25:42,None,5.0,2021-01-03 00:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 10:22:27,2021-03-01 10:22:27,None,2021-03-01 10:22:28,None,25.0,2020-08-17 00:00:00,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,603d0b710a720fde1000042a,NaN,NaN,2021-03-01 15:42:41,2021-03-01 15:42:41,None,2021-03-01 15:42:41,None,NaN,None,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,603cf5290a720fde10000413,NaN,NaN,2021-03-01 14:07:37,2021-03-01 14:07:37,None,2021-03-01 14:07:37,None,NaN,None,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28,2021-03-01 13:07:28,None,2021-03-01 13:07:29,None,25.0,2020-08-17 00:00:00,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [32]:
df_receipts

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,None,2021-01-03 15:25:42,None,5.0,2021-01-03 00:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 10:22:27,2021-03-01 10:22:27,None,2021-03-01 10:22:28,None,25.0,2020-08-17 00:00:00,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,603d0b710a720fde1000042a,NaN,NaN,2021-03-01 15:42:41,2021-03-01 15:42:41,None,2021-03-01 15:42:41,None,NaN,None,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,603cf5290a720fde10000413,NaN,NaN,2021-03-01 14:07:37,2021-03-01 14:07:37,None,2021-03-01 14:07:37,None,NaN,None,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28,2021-03-01 13:07:28,None,2021-03-01 13:07:29,None,25.0,2020-08-17 00:00:00,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


#### Looks like the column rewardsReceiptItemList has information contained within lists and dictionaries, so we will have to extract that information out and place it in its own columns.

In [33]:
df_receipts.rewardsReceiptItemList[0]

[{'barcode': '4011',
  'description': 'ITEM NOT FOUND',
  'finalPrice': '26.00',
  'itemPrice': '26.00',
  'needsFetchReview': False,
  'partnerItemId': '1',
  'preventTargetGapPoints': True,
  'quantityPurchased': 5,
  'userFlaggedBarcode': '4011',
  'userFlaggedNewItem': True,
  'userFlaggedPrice': '26.00',
  'userFlaggedQuantity': 5}]

In [34]:
#User previous Github Repo for help!
df_receipts = df_receipts.explode('rewardsReceiptItemList')
df_receipts.reset_index(inplace=True)
df_receipts = df_receipts.fillna({'rewardsReceiptItemList':'{}'})
df_receipts['rewardsReceiptItemList'] = df_receipts['rewardsReceiptItemList'].apply(lambda x:str(x))
df_receipts['rewardsReceiptItemList'] = df_receipts['rewardsReceiptItemList'].apply(literal_eval)
    
    

In [35]:
df_receipts['rewardsReceiptItemList'][0]

{'barcode': '4011',
 'description': 'ITEM NOT FOUND',
 'finalPrice': '26.00',
 'itemPrice': '26.00',
 'needsFetchReview': False,
 'partnerItemId': '1',
 'preventTargetGapPoints': True,
 'quantityPurchased': 5,
 'userFlaggedBarcode': '4011',
 'userFlaggedNewItem': True,
 'userFlaggedPrice': '26.00',
 'userFlaggedQuantity': 5}

In [36]:
df_receipts['rewardsReceiptItemList_barcode'] = df_receipts['rewardsReceiptItemList'].apply(lambda x: x.get('barcode'))
df_receipts['rewardsReceiptItemList_description'] = df_receipts['rewardsReceiptItemList'].apply(lambda x: x.get('description'))
df_receipts['rewardsReceiptItemList_finalPrice'] = df_receipts['rewardsReceiptItemList'].apply(lambda x: x.get('finalPrice'))
df_receipts['rewardsReceiptItemList_itemPrice'] = df_receipts['rewardsReceiptItemList'].apply(lambda x: x.get('itemPrice'))
df_receipts['rewardsReceiptItemList_needsFetchReview'] = df_receipts['rewardsReceiptItemList'].apply(lambda x: x.get('needsFetchReview'))
df_receipts['rewardsReceiptItemList_partnerItemId'] = df_receipts['rewardsReceiptItemList'].apply(lambda x: x.get('partnerItemId'))
df_receipts['rewardsReceiptItemList_preventTargetGapPoints'] = df_receipts['rewardsReceiptItemList'].apply(lambda x: x.get('preventTargetGapPoints'))
df_receipts['rewardsReceiptItemList_quantityPurchased'] = df_receipts['rewardsReceiptItemList'].apply(lambda x: x.get('quantityPurchased'))
df_receipts['rewardsReceiptItemList_userFlaggedBarcode'] = df_receipts['rewardsReceiptItemList'].apply(lambda x: x.get('userFlaggedBarcode'))
df_receipts['rewardsReceiptItemList_userFlaggedNewItem'] = df_receipts['rewardsReceiptItemList'].apply(lambda x: x.get('userFlaggedNewItem'))
df_receipts['rewardsReceiptItemList_userFlaggedPrice'] = df_receipts['rewardsReceiptItemList'].apply(lambda x: x.get('userFlaggedPrice'))
df_receipts['rewardsReceiptItemList_userFlaggedQuantity'] = df_receipts['rewardsReceiptItemList'].apply(lambda x: x.get('userFlaggedQuantity'))

In [37]:
df_receipts.drop('rewardsReceiptItemList', axis=1, inplace = True)

In [38]:
df_receipts.head()
df_receipts.columns

Index(['index', '_id', 'bonusPointsEarned', 'bonusPointsEarnedReason',
       'createDate', 'dateScanned', 'finishedDate', 'modifyDate',
       'pointsAwardedDate', 'pointsEarned', 'purchaseDate',
       'purchasedItemCount', 'rewardsReceiptStatus', 'totalSpent', 'userId',
       'rewardsReceiptItemList_barcode', 'rewardsReceiptItemList_description',
       'rewardsReceiptItemList_finalPrice', 'rewardsReceiptItemList_itemPrice',
       'rewardsReceiptItemList_needsFetchReview',
       'rewardsReceiptItemList_partnerItemId',
       'rewardsReceiptItemList_preventTargetGapPoints',
       'rewardsReceiptItemList_quantityPurchased',
       'rewardsReceiptItemList_userFlaggedBarcode',
       'rewardsReceiptItemList_userFlaggedNewItem',
       'rewardsReceiptItemList_userFlaggedPrice',
       'rewardsReceiptItemList_userFlaggedQuantity'],
      dtype='object')

## Data Quality Issues

#### While cleaning the data I noticed missing values, so lets explore those

In [39]:
# Seems like the colummns lastLogin, signUpSource, and state all have missing values.
df_users.isnull().sum()

_id              0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64

In [40]:
#Many of the columns have missing values here.
df_receipts.isnull().sum()

index                                               0
_id                                                 0
bonusPointsEarned                                1401
bonusPointsEarnedReason                          1401
createDate                                          0
dateScanned                                         0
finishedDate                                     1411
modifyDate                                          0
pointsAwardedDate                                1301
pointsEarned                                     1128
purchaseDate                                      458
purchasedItemCount                                484
rewardsReceiptStatus                                0
totalSpent                                        435
userId                                              0
rewardsReceiptItemList_barcode                   4291
rewardsReceiptItemList_description                821
rewardsReceiptItemList_finalPrice                 614
rewardsReceiptItemList_itemP

In [41]:
#These columns have missing values as well.
df_brands.isnull().sum()

_id               0
barcode           0
category        155
categoryCode    650
cpg               0
name              0
topBrand        612
brandCode       234
cpg_id            0
cpg_ref           0
dtype: int64

In [42]:
# Users dataframe contain duplicates which can be problem during analysis.
df_receipts['_id'].duplicated().any()

True

In [43]:
# Users dataframe contain duplicates which can be problem during analysis.
df_users['_id'].duplicated().any()

True